# A Guide On Solving Non-Convex Consumption-Saving Models - 2D

This notebook produces the timing and accuracy results for the *extended* benchmark model *with two durable stocks* in [A Guide to Solve Non-Convex Consumption-Saving Models](https://drive.google.com/open?id=1V15dwMIrl_TJGoqu7qauhVWcDm0yqb-D).

# Setup

In [ ]:
from consav import runtools
runtools.write_numba_config(disable=0,threads=8)

In [ ]:
import time
import itertools as it
import numpy as np
from copy import copy

%matplotlib inline

# reload module each time cell is run
%load_ext autoreload
%autoreload 2

# load the DurableConsumptionModel module
from DurableConsumptionModel import DurableConsumptionModelClass

# Timing function

In [ ]:
def run(name = 'baseline',
        load = False,
        solmethod = 'nvfi_cpp_2d',
        updpar = {},
        do_print = False):
    
    # a. create model
    model = DurableConsumptionModelClass(name=name,load=load,solmethod=solmethod,
                                         do_print=do_print,**updpar)

    # b. run
    if load == False:
              
        # i. solve
        model.solve()        
        
        total_time = np.sum(model.par.time_w+model.par.time_keep+model.par.time_adj)
        print(f'total time = {total_time:.2f}')
                
        # ii. simulate
        model.simulate(do_euler_error=True,do_utility=True)
    
        # iii. save
        model.save()

    # c. print progress
    print('')    
    model.print_analysis()        
    model.checksum(simple=True)
    print('')
    
    return model

# Settings

In [ ]:
load = False
load_negm_cpp_2d = load
load_nvfi_cpp_2d = load
load_vfi_cpp_2d = load

T = 50
models = dict()

# baseline
basename = 'results'
base = dict()
base['do_2d'] = True
base['n_max'] = 2.0
base['Np'] = 50
base['Nn'] = 50
base['Nm'] = 100
base['Nx'] = 100
base['Na'] = 100
base['T'] = T

# Timings

In [ ]:
models['negm_2d_cpp'] = run(name=f'{basename}',load=load_negm_cpp_2d,solmethod='negm_2d_cpp',updpar=base)
models['nvfi_2d_cpp'] = run(name=f'{basename}',load=load_nvfi_cpp_2d,solmethod='nvfi_2d_cpp',updpar=base)
models['vfi_2d_cpp'] = run(name=f'{basename}',load=load_vfi_cpp_2d,solmethod='vfi_2d_cpp',updpar=base)

# Tables

In [ ]:
import tabs

In [ ]:
main_models = [models['vfi_2d_cpp'],models['nvfi_2d_cpp'],models['negm_2d_cpp']]
tabs.all(main_models,speedup=True,postfix='_2d')

# Figures

In [ ]:
from figs import lifecycle_compare

In [ ]:
lifecycle_compare(models['negm_2d_cpp'],'NEGM',models['nvfi_2d_cpp'],'NVFI',do_euler_errors=True)

In [ ]:
lifecycle_compare(models['negm_2d_cpp'],'NEGM',models['vfi_2d_cpp'],'VFI',do_euler_errors=True)